In [ ]:
import os

# Download and extract data

In [12]:
LINK_TO_DATA='1s1svxg4CvyUi9eeMzqUbcpiwCtNc170z'

In [3]:
!pip install gdown

/home/abuzarmahmood/anaconda3/lib/python3.6/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/home/abuzarmahmood/anaconda3/lib/python3.6/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [4]:
!mkdir test_data

In [5]:
os.chdir('./test_data')

In [13]:
!gdown {LINK_TO_DATA} -O test_data.zip

Downloading...
From: https://drive.google.com/uc?id=1s1svxg4CvyUi9eeMzqUbcpiwCtNc170z
To: /media/bigdata/projects/pytau/pytau/how_to/examples/test_data/test_data.zip
100%|████████████████████████████████████████| 514k/514k [00:00<00:00, 16.0MB/s]


In [15]:
!unzip test_data.zip

Archive:  test_data.zip
replace AM11_4Tastes_191030_114043_spike_trains_only/AM11_4Tastes_191030_114043_copy.info? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [16]:
!rm test_data.zip

rm: cannot remove 'test_data.zip': No such file or directory


In [17]:
data_dir = os.path.abspath('AM11_4Tastes_191030_114043_spike_trains_only')
print(data_dir)

# Deal with paths

In [21]:
pytau_base_dir = '/media/bigdata/projects/pytau'
sys.path.append(pytau_base_dir)

In [23]:
# Change database save path
!echo {pytau_base_dir}/pytau/how_to/examples/saved_models > {pytau_base_dir}/pytau/config/MODEL_SAVE_DIR.params

# Perform Analyses

In [42]:
## Import modules
import sys
from pytau.changepoint_io import FitHandler, DatabaseHandler

In [37]:
# Specify params for fit
model_parameters = dict(
    states=4,
    fit=40000,
    samples=20000,
    model_kwargs={'None': None},
        )

preprocess_parameters = dict(
    time_lims=[2000, 4000],
    bin_width=50,
    data_transform='None',  # Can also be 'spike_shuffled','trial_shuffled'
    )

FitHandler_kwargs = dict(
    data_dir=data_dir,
    taste_num= 0,
    region_name= 'all',  # Should match specification in info file or 'all'
    laser_type=None,
    experiment_name= 'pytau_test',
    )

In [38]:
## Initialize handler, and feed paramters
handler = FitHandler(**FitHandler_kwargs)
handler.set_model_params(**model_parameters)
handler.set_preprocess_params(**preprocess_parameters)

Fit database does not exist yet
MODEL_PARAMS will have to be set
PREPROCESS_PARAMS will have to be set


In [39]:
handler.run_inference()
handler.save_fit_output()

Loading spike trains from AM11_4Tastes_191030_114043_spike_trains_only, dig_in 0, laser None
Preprocessing spike trains, preprocessing func: <preprocess_single_taste>
Generating Model, model func: <single_taste_poisson>
Running inference, inference func: <advi_fit>


Finished [100%]: Average Loss = 22,166


Saving inference output to : 
/media/bigdata/projects/pytau/pytau/how_to/examples/saved_models/pytau_test



### Look at inference outputs

In [41]:
handler.inference_outs.keys()

dict_keys(['model', 'approx', 'lambda', 'tau', 'data'])

# Load data from saved output

In [43]:
fit_database = DatabaseHandler()
fit_database.drop_duplicates()
fit_database.clear_mismatched_paths()

0 rows found with all NA, removing...
Removing 0 duplicate rows
0 mismatches from database
0 mismatches from files
==== Clearing Completed ====


In [45]:
dframe = fit_database.fit_database

In [47]:
dframe.T

,0
preprocess.time_lims,"[2000, 4000]"
preprocess.bin_width,50
preprocess.data_transform,None
preprocess.preprocessor_name,preprocess_single_taste
model.states,4
model.fit,40000
model.samples,20000
model.model_kwargs,{'None': None}
model.model_template_name,single_taste_poisson
model.inference_func_name,advi_fit


In [54]:
# Load data
from pytau.changepoint_analysis import PklHandler
exp_save_path = dframe['exp.save_path'].iloc[0]
this_handler = PklHandler(exp_save_path)

/home/abuzarmahmood/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [55]:
dir(this_handler)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_fit_model',
 '_model_structure',
 'data',
 'dir_name',
 'file_name_base',
 'firing',
 'lambda_array',
 'metadata',
 'pkl_file_path',
 'pretty_metadata',
 'processed_spikes',
 'tau',
 'tau_array']

In [56]:
this_handler.tau.scaled_mode_tau

array([[2747, 3674, 3761],
       [2848, 2863, 2905],
       [2498, 3317, 3351],
       [3028, 3053, 3075],
       [3099, 3200, 3941],
       [2149, 2719, 2780],
       [2557, 3061, 3171],
       [2548, 2892, 3075],
       [2389, 3085, 3196],
       [2654, 3451, 3627],
       [2252, 3055, 3137],
       [2248, 2965, 3053],
       [2476, 2545, 2670],
       [2083, 3502, 3947],
       [2766, 2951, 3937],
       [2484, 3063, 3947],
       [2236, 2347, 3383],
       [2476, 2597, 3945],
       [2433, 2788, 3531],
       [2362, 2459, 3905],
       [2112, 2626, 3740],
       [2372, 2910, 3947],
       [2068, 2660, 3157],
       [2046, 3325, 3475],
       [2939, 3051, 3714],
       [2427, 2896, 2974],
       [2083, 3152, 3200],
       [2056, 3592, 3825],
       [2038, 3092, 3939],
       [2041, 3848, 3946]])